In [ ]:
import altair as alt
import polars as pl
from plot_data import month_order
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

from iup.utils import date_to_season

In [49]:
SEASON_START_MONTH = 7  # July
END_MONTH = 9  # 9 months after July, i.e., April
months = month_order(season_start_month=SEASON_START_MONTH)

data = (
    pl.read_parquet("../data/raw.parquet")
    .filter(pl.col("geography_type") == pl.lit("admin1"))
    .with_columns(
        season=date_to_season(
            pl.col("time_end"), season_start_month=SEASON_START_MONTH
        ),
        t=pl.col("time_end")
        .dt.to_string("%b")
        .map_elements(lambda x: months.index(x) - END_MONTH, pl.Int64),
    )
    .filter(pl.col("t") <= 0)
    # remove partial seasons
    .filter(pl.col("season").is_in(["2008/2009", "2022/2023"]).not_())
    .select(["season", "geography", "t", "estimate"])
    .sort(["season", "geography", "t"])
)

data

season,geography,t,estimate
str,str,i64,f64
"""2009/2010""","""Alabama""",-8,0.007
"""2009/2010""","""Alabama""",-7,0.082
"""2009/2010""","""Alabama""",-6,0.255
"""2009/2010""","""Alabama""",-5,0.334
"""2009/2010""","""Alabama""",-4,0.354
…,…,…,…
"""2021/2022""","""Wyoming""",-4,0.352
"""2021/2022""","""Wyoming""",-3,0.36
"""2021/2022""","""Wyoming""",-2,0.37


In [53]:
rf = RandomForestRegressor(oob_score=True)
enc = OrdinalEncoder()

features = ["season", "geography", "t"]
X = data.select(features).to_numpy()
y = data.select("estimate").to_series().to_numpy()

enc.fit(X)
rf.fit(enc.transform(X), y)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [54]:
print(*zip(features, rf.feature_importances_))

('season', np.float64(0.0303123959134705)) ('geography', np.float64(0.07345371535177032)) ('t', np.float64(0.8962338887347592))


In [ ]:
# post hoc (but OOB) comparison of end-of-season predictions
alt.Chart(
    data.with_columns(pred=rf.oob_prediction_).filter(pl.col("t") == 0)
).mark_point().encode(alt.X("estimate"), alt.Y("pred"))

alt.Chart(...)

In [ ]:
# distribution of end-of-season errors
alt.Chart(
    data.with_columns(pred=rf.oob_prediction_)
    .filter(pl.col("t") == 0)
    .with_columns(error=pl.col("pred") - pl.col("estimate"))
).mark_bar().encode(alt.X("error", bin=True), alt.Y("count()"))

alt.Chart(...)

In [95]:
def forecast(
    forecast_t: int, target_t: int = 0, target_season: str = "2021/2022", data=data
):
    features = ["season", "geography", "t"]

    data_fit = data.filter(
        (
            (pl.col("t") > forecast_t) & (pl.col("season") == pl.lit(target_season))
        ).not_()
    )
    data_pred = data.select(pl.col("geography").unique()).with_columns(
        season=pl.lit(target_season), t=target_t
    )

    enc = OrdinalEncoder()
    enc.fit(data.select(features).to_numpy())

    X_fit = data_fit.select(features).to_numpy()
    y_fit = data_fit.select("estimate").to_series().to_numpy()

    rf = RandomForestRegressor()
    rf.fit(enc.transform(X_fit), y_fit)

    # make the forecast
    X_pred = data_pred.select(features).to_numpy()
    y_pred = rf.predict(enc.transform(X_pred))

    return data_pred.with_columns(forecast_t=forecast_t, pred=y_pred).drop(
        ["season", "t"]
    )


forecasts = pl.concat([forecast(x) for x in [-10, -8, -5, -3, -1]])
forecasts

geography,forecast_t,pred
str,i32,f64
"""Iowa""",-10,0.53224
"""Colorado""",-10,0.53981
"""Alaska""",-10,0.47034
"""Vermont""",-10,0.58686
"""Missouri""",-10,0.47202
…,…,…
"""Washington""",-1,0.51844
"""Wisconsin""",-1,0.4937
"""Ohio""",-1,0.45358


In [96]:
# join in the true values
forecast_target = (
    data.filter(pl.col("season") == pl.col("season").max(), pl.col("t") == 0)
    .drop(["season", "t"])
    .with_columns(forecast_t=0)
    .rename({"estimate": "pred"})
)

(
    alt.Chart(pl.concat([forecasts, forecast_target], how="diagonal"))
    .mark_line()
    .encode(alt.X("forecast_t"), alt.Y("pred"), alt.Detail("geography"))
)

alt.Chart(...)